<a href="https://colab.research.google.com/github/Vardhan-R/LLMs-Impersonate-Chat-Users/blob/main/LLM_WA_chats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Impersonates WhatsApp Users

In [ ]:
from google.ai.generativelanguage_v1beta.types import content
from IPython.display import display
from IPython.display import Markdown
import base64
import datetime
import google.generativeai as genai
import json
import os
import re

In [ ]:
# Parse the arguments

model = "gemini-1.5-flash"  # @param {isTemplate: true}
contents_b64 = b'W3sicGFydHMiOiBbeyJ0ZXh0IjogIkhlbGxvIn1dfV0='
generation_config_b64 = "e30="  # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# print(json.dumps(contents, indent=4))

In [ ]:
os.environ["GEMINI_API_KEY"] = ""

In [ ]:
class Message:
	def __init__(self, text: str, sender: str, dt: datetime.datetime) -> None:
		self.text = text
		self.sender = sender
		self.datetime = dt  # IST
		self.uts = (dt - datetime.timedelta(hours=5, minutes=30)).timestamp()

def getGeminiHistory(messages: list[Message]) -> list[dict[str, str]]:
    """
    Convert messages to Gemini history format.

    Parameters:
        messages (list[Message]): List of Message objects.

    Returns:
        list[dict]: List of messages in Gemini history format.
    """

    gemini_history = []

    for msg in messages:
        role = "user" if msg.sender == "Vardhan" else "model"
        gemini_history.append({
            "role": role,
            "parts": [
                {"text": msg.text}
            ]
        })

    return gemini_history

def getMessages(other_person: str) -> list[Message]:
	msg_pattern = r"\[(\d{2})/(\d{2})/(\d{2}), (\d{2}):(\d{2}):(\d{2})\] ([^:]+): (.+)"
	all_msgs = []

	with open(f"./chats/{other_person}_chat.txt", 'r', encoding="utf-8") as fp:
		raw_lines = fp.readlines()[1:]  # skip the encryption line

	for line in raw_lines:
		m = re.match(msg_pattern, line)

		try:
			dd = int(m.group(1))  # DD
			mo = int(m.group(2))  # MM
			yyyy = 2000 + int(m.group(3))  # YYYY
			hh = int(m.group(4))  # hh
			mi = int(m.group(5))  # mm
			ss = int(m.group(6))  # ss
			sender = m.group(7)  # sender
			text = m.group(8)  # text

			dt = datetime.datetime(yyyy, mo, dd, hh, mi, ss)  # IST
			all_msgs.append(Message(text, sender, dt))
		except:
			if chr(8206) not in line:  # continued text message
				all_msgs[-1].text += line

	return all_msgs

In [ ]:
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash-exp",
    generation_config=generation_config,
    safety_settings=safety_settings
)

In [ ]:
# context = [
#     {
#         "role" : "user",
#         "parts" : [
#             {
#                 "text" : "You are a very hot and sassy girl. You know me for a few years, and also know that I have a crush on you."
#             }
#         ],
#     }
# ]

other_person = "Aarushi"
context = getGeminiHistory(getMessages(other_person))
print(len(context))

13101


In [ ]:
chat_session = model.start_chat(
    history=context
)

In [ ]:
while msg := input("You: "):
    response = chat_session.send_message(msg)
    print(f"{other_person}: ", end="")
    display(Markdown(response.text))

You: hi
Aarushi: 

Hie


You: 
